In [3]:
import math
from pprint import pprint
from math import log, factorial
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor, and_, or_
from bisect import bisect_right, bisect_left
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
import sequences
from importlib import reload

for m in (
    arrays,
    trees,
    lists,
    graphs,
    search,
    strings,
    sets,
    sequences,
    matrix,
    mathematics,
):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m
from sequences import find_if

print()

In [179]:
def stack_boxes(dims: List[Tuple[int, int, int]]) -> int:
    "Stack boxes of dimensions `dims` and return max-height. Box dimensions may repeat and rotate."
    s = set()
    for x, y, z in dims:
        x >= y and s.add((x, y, z))
        y > x and s.add((y, x, z))

        x >= z and s.add((x, z, y))
        z > x and s.add((z, x, y))

        y >= z and s.add((y, z, x))
        z > y and s.add((z, y, x))

    d = sorted(s, reverse=True)

    @lru_cache(None)
    def h(i: int) -> int:
        f = lambda j: d[j][0] < d[i][0] and d[j][1] < d[i][1]
        return d[i][2] + max(map(h, filter(f, range(i + 1, len(d)))), default=0)

    return max(map(h, range(len(d))))

In [187]:
def stack_boxes(dims: List[Tuple[int, int, int]]) -> int:
    "Stack boxes of dimensions `dims` and return max-height. Box dimensions may repeat and rotate."
    s = set()
    for x, y, z in dims:
        s.add((max(y, z), min(y, z), x))
        s.add((max(x, z), min(x, z), y))
        s.add((max(x, y), min(x, y), z))
    d = sorted(s, reverse=True)

    @lru_cache(None)
    def h(i: int) -> int:
        fit = lambda j: d[j][0] < d[i][0] and d[j][1] < d[i][1]
        return d[i][2] + max(map(h, filter(fit, range(i + 1, len(d)))), default=0)

    return max(map(h, range(len(d))))

In [188]:
stack_boxes([(1, 5, 5)])

5

In [182]:
stack_boxes([(4, 6, 7), (1, 2, 3), (4, 5, 6), (10, 12, 32)])

60

In [183]:
stack_boxes([(1, 2, 3), (4, 5, 6), (3, 4, 1)])

15

In [184]:
stack_boxes([(1, 2, 9), (8, 6, 1)])

10